In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import time
import datetime

In [2]:
features = pd.read_csv('features.csv', index_col='match_id')
features.drop(['tower_status_radiant',
               'tower_status_dire',
               'barracks_status_radiant',
               'barracks_status_dire',
               'duration'], axis=1, inplace=True)

# Логистическая регрессия

 ### Предобработка данных

``Пропуски в таблице заполним нулями``

In [3]:
features_0 = features.copy(deep = True)
for name in list(features_0.columns.values):
    features_0[name].fillna(value=0, inplace=True)

In [4]:
X_0 = features_0.loc[:,features_0.columns != 'radiant_win']
y = pd.DataFrame(features['radiant_win'])

`Произведем скалирование полей`

In [5]:
scaler = StandardScaler()
X_0_scaled = pd.DataFrame(scaler.fit_transform(X_0))

/home/ivan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ivan/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


#### Проведение кросс-валидации

`Функция, которая производит кросс-валидацию для логистической регрессии, а так же выбирает оптимальное значение C`

In [6]:
def logreg_cv_with_auc_roc_value(X_0, y, start, end, n_step):
    auc_roc_list = []
    timing_list = []
    kf = KFold(n_splits=5 ,random_state = 241, shuffle=True)
    for c in np.linspace(start, end, n_step):
        roc_auc_score_list = []
        start_time = datetime.datetime.now()
        clf = LogisticRegression(solver = 'saga',random_state = 241, penalty='l2', C=c, max_iter = 1000)
        for train, test in kf.split(X_0, y):
            X_0_train = X_0.iloc[train, :].values.tolist()
            X_0_test = X_0.iloc[test, :].values.tolist()
            y_train = list(map(lambda x: x.pop(), y.iloc[train, :].values.tolist()))
            y_test = list(map(lambda x: x.pop(),y.iloc[test, :].values.tolist()))
            probs = clf.fit(X_0_train, y_train).predict_proba(X_0_test)[:, 1]
            roc_auc_value = roc_auc_score(y_test, probs)
            roc_auc_score_list.append(roc_auc_value)
        delta_time = datetime.datetime.now() - start_time
        auc_roc_list.append(np.mean(roc_auc_score_list))
        timing_list.append(delta_time)
        #print(f'AUC-ROC value with C coef equal {c} is {np.mean(roc_auc_score_list)} with elapsed time {delta_time}')
    print(f'Best AUC-ROC value is {max(auc_roc_list)} with C value {(auc_roc_list.index(max(auc_roc_list))+1)*(end/n_step)}')
    print(f'Time elapsed {timing_list[auc_roc_list.index(max(auc_roc_list))]}')
    return max(auc_roc_list), min(auc_roc_list)

#### Подбор оптимального параметра C

`Наибольшее значение AUC-ROC параметра находится на краю заданного отрезка, посмотрим, какое оптимальное значение на отрезке от 0.0001 до 0.005`

In [67]:
 auc_roc_value_1 = logreg_cv_with_auc_roc_value(X_0_scaled, y, 0.005, 1, 200)

Best AUC-ROC value is 0.7163568585900241 with C value 0.005
Time elapsed 0:00:15.709695


`Оптимальное значение C равно 0.0037`

In [71]:
 auc_roc_value_1 = logreg_cv_with_auc_roc_value(X_0_scaled, y, 0.0001, 0.005, 50)

Best AUC-ROC value is 0.7163600080198889 with C value 0.0037
Time elapsed 0:00:14.750661


#### Подбор параметра C для случая, когда в матрице признаков отсутсвуют категориальные признаки

`Удалим категориальные признаки`

In [7]:
X_0_without_cat_fields =X_0.drop(['lobby_type',
                                  'r1_hero',
                                  'r2_hero',
                                  'r3_hero',
                                  'r4_hero',
                                  'r5_hero',
                                  'd1_hero',
                                  'd2_hero',
                                  'd3_hero',
                                  'd4_hero',
                                  'd5_hero'], axis=1, inplace=False)
y = pd.DataFrame(features['radiant_win'])

In [8]:
scaler = StandardScaler()
X_0_without_cat_fields_scaled = pd.DataFrame(scaler.fit_transform(X_0_without_cat_fields))

/home/ivan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ivan/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


`Заметим, что после удаления категориальных признаков значение параметра AUC-ROC почти не изменилось`

In [74]:
auc_roc_value_2 = logreg_cv_with_auc_roc_value(X_0_without_cat_fields_scaled, y, 0.005, 1, 200)

Best AUC-ROC value is 0.7164131326312374 with C value 0.005
Time elapsed 0:01:10.652533


In [75]:
auc_roc_value_2 = logreg_cv_with_auc_roc_value(X_0_without_cat_fields_scaled, y, 0.0001, 0.005, 50)

Best AUC-ROC value is 0.7164150563086176 with C value 0.0036000000000000003
Time elapsed 0:00:11.931015


#### Подбор параметра C для случая, когда в матрице признаков учитываются категориальные признаки

`Добавим категориальные признаки и с помощью методики мешка слов преобразуем данные поля`

In [9]:
X_0_with_cat_fields_scaled = X_0_without_cat_fields_scaled.copy(deep = True)
lobby_indexes = list(set(X_0['lobby_type']))
for x in lobby_indexes:
    X_0_with_cat_fields_scaled[f'lobby{x}']= np.zeros((X_0.shape[0], 1))
for i, res in enumerate(X_0['lobby_type']):
    X_0_with_cat_fields_scaled[f'lobby{res}'][i]= 1

In [10]:
lst = []
names_heroes = ['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
for name in names_heroes:
    lst.append(list(set(X_0[name])))
heroes_indexes = list(set([y for x in lst for y in x]))
for x in heroes_indexes:
    X_0_with_cat_fields_scaled[f'hero{int(x)}']= np.zeros((X_0.shape[0], 1))

    
for index, match_id in enumerate(X_0.index):
    for p in range(1,6):
        r_row = int(X_0[f'r{p}_hero'][match_id])
        d_row = int(X_0[f'd{p}_hero'][match_id])
        X_0_with_cat_fields_scaled[f'hero{r_row}'][index]= 1
        X_0_with_cat_fields_scaled[f'hero{d_row}'][index]= -1

In [11]:
auc_roc_value_3 = logreg_cv_with_auc_roc_value(X_0_with_cat_fields_scaled, y, 0.005, 1, 200)

Best AUC-ROC value is 0.7520978166635774 with C value 0.05
Time elapsed 0:03:23.137500


In [12]:
auc_roc_value_3_1 = logreg_cv_with_auc_roc_value(X_0_with_cat_fields_scaled, y, 0.0001, 0.005, 50)

Best AUC-ROC value is 0.7513607241908662 with C value 0.005
Time elapsed 0:00:36.168811


`Значение параметра AUC-ROC увеличилось до 0.75`

#### Предобработка и обучение лучшей модели на тестовой выборке

`Далее идет преобразование тестовой выборки, обучение наилучшей модели на полной выборке и нахождение ответов`

In [27]:
features_test = pd.read_csv('features_test.csv', index_col='match_id')

In [28]:
for name in list(features_test.columns.values):
    features_test[name].fillna(value=0, inplace=True)

In [29]:
X_test_without_cat_fields =features_test.drop(['lobby_type',
                                  'r1_hero',
                                  'r2_hero',
                                  'r3_hero',
                                  'r4_hero',
                                  'r5_hero',
                                  'd1_hero',
                                  'd2_hero',
                                  'd3_hero',
                                  'd4_hero',
                                  'd5_hero'], axis=1, inplace=False)

In [30]:
scaler = StandardScaler()
X_test_without_cat_fields_scaled = pd.DataFrame(scaler.fit_transform(X_test_without_cat_fields))

/home/ivan/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ivan/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [31]:
X_test_with_cat_fields_scaled = X_test_without_cat_fields_scaled.copy(deep = True)
lobby_indexes = list(set(features_test['lobby_type']))
for x in lobby_indexes:
    X_test_with_cat_fields_scaled[f'lobby{x}']= np.zeros((features_test.shape[0], 1))
for i, res in enumerate(X_0['lobby_type']):
    X_test_with_cat_fields_scaled[f'lobby{res}'][i]= 1

In [32]:
lst = []
names_heroes = ['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
for name in names_heroes:
    lst.append(list(set(features_test[name])))
heroes_indexes = list(set([y for x in lst for y in x]))
for x in heroes_indexes:
    X_test_with_cat_fields_scaled[f'hero{int(x)}']= np.zeros((features_test.shape[0], 1))

    
for index, match_id in enumerate(features_test.index):
    for p in range(1,6):
        r_row = int(features_test[f'r{p}_hero'][match_id])
        d_row = int(features_test[f'd{p}_hero'][match_id])
        X_test_with_cat_fields_scaled[f'hero{r_row}'][index]= 1
        X_test_with_cat_fields_scaled[f'hero{d_row}'][index]= -1

In [35]:
clf = LogisticRegression(solver = 'saga',random_state = 241, penalty='l2', C=0.05, max_iter = 1000)

In [36]:
y_train = list(map(lambda x: list(x).pop(), y.values))

In [37]:
Y_test_proba = clf.fit(X_0_with_cat_fields_scaled.values.tolist(), y_train).predict_proba(X_test_with_cat_fields_scaled.values.tolist())[:, 1]

`Минимальное значение вероятности для тестовой выборке`

In [50]:
min(list(Y_test_proba))

0.009429917441092055

`Максимальное значение вероятности для тестовой выборке`

In [51]:
max(list(Y_test_proba))

0.9966216509212199

In [46]:
features_result = pd.DataFrame({'match_id': features_test.index, 'radiant_win': Y_test_proba})
features_result.to_csv('result_out.csv',index=False)

## Резюме по логистической регрессии 

Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

 - Качество логистической регрессии для обучающей выборки на кросс-валидации получилось равным 0.716, что немного больше, чем 0.69 от градиентного бустинга. 
 - Разницу можно объяснить небольшим количеством деревьев в градиентном бустинге. 
 - Логистическая регрессия работает быстрее градиентного бустинга. (15 секунд против 1.5 минуты)

Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

- Качество практически повышается на очень малую величину (0.71636 против 0.7164)
- Можно объяснить тем, что категориальные признаки, которые воспринималются, как количественные, ухудшают качество обучения  

Сколько различных идентификаторов героев существует в данной игре?

- 108

Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

- Качество при добавлении "мешка слов" по героям увеличилось до 0.75. Улучшилось. 
- Объяснить можно тем, что при правильном внесении категориальных признаков в матрицу признаков, обучающий алгоритм получает гораздо больше информации, а следовательно можно лучше предсказывать результаты тестовой выборки

Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

- Минимальное - 0.009429917441092055

- Максимальное - 0.9966216509212199